### AI agents are the programs where the LLMs control the workflow.

#### AI Agent: 
#### → Goal-oriented reasoning 
#### → Decides its own next steps 
#### → Adapts to unexpected situations 
#### → Actually thinks through problems

In [1]:
from agents import Agent, Runner 
import loadenv 